In [1]:
# import mpl_finance
# import matplotlib.dates as mdates 
from talib import MA,STOCH
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
START_OF_2018 = 6044
START_OF_2017 = 5793
TIME_STEP = 30

In [3]:
# big_four = df[["Date", "Open", "High", "Low", "Close"]] 6046 -> 6044 (2018)
# big_four_2018 = big_four[START_OF_2018:] 5795 -> 5793 (2017)
# ax = plt.subplot()
# mpl_finance.candlestick_ohlc(ax, big_four_2018, width=5, colorup='g', colordown='r')
# ax.xaxis_date()
# ax.grid(True)
# plt.show()
# date_2018 = df[START_OF_2018:]["Date"]

In [4]:
df = pd.read_csv("SPY.csv",date_parser=True)

In [5]:
df_train = df[ df["Date"]<'2017-01-01' ].copy()
# df_validation = df[(df['Date'] >= '2017-01-01') & (df['Date'] < "2018-01-01")].copy()
df_validation = df[df["Date"] >= "2017-01-01"].copy()
df_plot = df[df["Date"] >= "2018-01-01"].copy()
# df_validation.tail()

# Graph Plotting

In [6]:
# ma_10_2018 = MA(df[START_OF_2018 - 10:]["Close"], timeperiod=10, matype=0)
# ma_30_2018 = MA(df[START_OF_2018 - 30:]["Close"], timeperiod=30, matype=0)
# df_2018_neg8 = df[START_OF_2018-8:]
# k_2018, d_2018 = STOCH(df_2018_neg8["High"], df_2018_neg8["Low"], df_2018_neg8["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# plt.plot(np.arange(len(ma_10_2018) - 10), ma_10_2018[10:], label="MA-10")
# plt.plot(np.arange(len(ma_30_2018) - 30), ma_30_2018[30:], label="MA-30")
# plt.legend()
# plt.show()

# Add Technical Analysis

In [7]:
ma_10_train = MA(df_train["Close"], timeperiod=10, matype=0)
ma_30_train = MA(df_train["Close"], timeperiod=30, matype=0)
k_train, d_train = STOCH(df_train["High"], df_train["Low"], df_train["Close"],fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_train["MA10"] = ma_10_train
df_train["MA30"] = ma_30_train
df_train["K"] = k_train
df_train["D"] = d_train

ma_10_validation = MA(df_validation["Close"], timeperiod=10, matype=0)
ma_30_validation = MA(df_validation["Close"], timeperiod=30, matype=0)
k_validation, d_validation = STOCH(df_validation["High"], df_validation["Low"], df_validation["Close"],fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df_validation["MA10"] = ma_10_validation
df_validation["MA30"] = ma_30_validation
df_validation["K"] = k_validation
df_validation["D"] = d_validation

In [8]:
df_validation.shape

(501, 11)

## Drop dates and NA

In [9]:
df_train = df_train.dropna(axis=0)
df_train = df_train.drop(["Date"], axis=1)

df_validation = df_validation.dropna(axis=0)
df_validation = df_validation.drop(["Date"], axis=1)

In [10]:
df_validation.shape

(472, 10)

## Normalize data

In [11]:
def normalizeDataframe(data_frame):
    normalize_df = data_frame.copy()
    for column in normalize_df.columns:
        min_value = min(normalize_df[column])
        max_value = max(normalize_df[column])
        normalize_df[column] = (normalize_df[column] - min_value) / (max_value - min_value)
    return normalize_df

In [12]:
df_train = normalizeDataframe(df_train)
df_validation = normalizeDataframe(df_validation)

In [13]:
df_validation.shape

(472, 10)

## Prepare X, y train and validation for RNN

In [14]:
data_train = df_train.values
data_validation = df_validation.values

In [15]:
X_train = []
y_train = []
X_validation = []
y_validation = []

In [16]:
for i in range(30,data_train.shape[0]):
    X_train.append(data_train[i-30:i])
    y_train.append(data_train[i, 0])

for i in range(30, data_validation.shape[0]):
    X_validation.append(data_validation[i-30:i])
    y_validation.append(data_validation[i,0])

In [17]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_validation, y_validation = np.array(X_validation), np.array(y_validation)

In [18]:
X_train.shape
# y_train.shape

(5734, 30, 10)

# Building LSTM

In [19]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [20]:
regressior = Sequential()
regressior.add(LSTM(units = 32, activation = 'tanh', input_shape = (X_train.shape[1], X_train.shape[2])))
regressior.add(Dense(units = 1))

In [21]:
regressior.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5504      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 5,537
Trainable params: 5,537
Non-trainable params: 0
_________________________________________________________________


In [22]:
checkpoint = ModelCheckpoint(filepath="best_params.hdf5", monitor="val_loss",verbose=1,save_best_only=True)

In [23]:
regressior.compile(optimizer='adam', loss = 'mean_squared_error')

In [24]:
regressior.fit(X_train, y_train, epochs=128, batch_size=64, validation_data = (X_validation, y_validation),callbacks=[checkpoint])

Train on 5734 samples, validate on 442 samples
Epoch 1/128
5568/5734 [============================>.] - ETA: 0s - loss: 0.0131
Epoch 00001: val_loss improved from inf to 0.00653, saving model to best_params.hdf5
5734/5734 [==============================] - 5s 851us/sample - loss: 0.0128 - val_loss: 0.0065
Epoch 2/128
5696/5734 [============================>.] - ETA: 0s - loss: 4.4498e-04
Epoch 00002: val_loss improved from 0.00653 to 0.00559, saving model to best_params.hdf5
5734/5734 [==============================] - 2s 398us/sample - loss: 4.4334e-04 - val_loss: 0.0056
Epoch 3/128
5632/5734 [============================>.] - ETA: 0s - loss: 2.6244e-04
Epoch 00003: val_loss improved from 0.00559 to 0.00500, saving model to best_params.hdf5
5734/5734 [==============================] - 2s 395us/sample - loss: 2.6177e-04 - val_loss: 0.0050
Epoch 4/128
5568/5734 [============================>.] - ETA: 0s - loss: 1.9222e-04
Epoch 00004: val_loss improved from 0.00500 to 0.00462, saving mo

Epoch 31/128
5696/5734 [============================>.] - ETA: 0s - loss: 3.8086e-05
Epoch 00031: val_loss improved from 0.00192 to 0.00182, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 440us/sample - loss: 3.7977e-05 - val_loss: 0.0018
Epoch 32/128
5696/5734 [============================>.] - ETA: 0s - loss: 4.1295e-05
Epoch 00032: val_loss did not improve from 0.00182
5734/5734 [==============================] - 3s 461us/sample - loss: 4.1119e-05 - val_loss: 0.0019
Epoch 33/128
5696/5734 [============================>.] - ETA: 0s - loss: 3.7657e-05
Epoch 00033: val_loss improved from 0.00182 to 0.00176, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 487us/sample - loss: 3.7759e-05 - val_loss: 0.0018
Epoch 34/128
5696/5734 [============================>.] - ETA: 0s - loss: 3.8365e-05- ETA: 0s - los
Epoch 00034: val_loss improved from 0.00176 to 0.00166, saving model to best_params.hdf5
5734/5734 [===================

Epoch 61/128
5696/5734 [============================>.] - ETA: 0s - loss: 1.7307e-05
Epoch 00061: val_loss improved from 0.00091 to 0.00088, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 469us/sample - loss: 1.7460e-05 - val_loss: 8.7998e-04
Epoch 62/128
5696/5734 [============================>.] - ETA: 0s - loss: 1.8536e-05
Epoch 00062: val_loss did not improve from 0.00088
5734/5734 [==============================] - 3s 460us/sample - loss: 1.8534e-05 - val_loss: 8.8964e-04
Epoch 63/128
5632/5734 [============================>.] - ETA: 0s - loss: 2.1066e-05
Epoch 00063: val_loss improved from 0.00088 to 0.00084, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 560us/sample - loss: 2.1289e-05 - val_loss: 8.4479e-04
Epoch 64/128
5696/5734 [============================>.] - ETA: 0s - loss: 2.1073e-05
Epoch 00064: val_loss improved from 0.00084 to 0.00082, saving model to best_params.hdf5
5734/5734 [======================

Epoch 91/128
5696/5734 [============================>.] - ETA: 0s - loss: 1.1381e-05
Epoch 00091: val_loss improved from 0.00047 to 0.00047, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 479us/sample - loss: 1.1337e-05 - val_loss: 4.6876e-04
Epoch 92/128
5696/5734 [============================>.] - ETA: 0s - loss: 1.0299e-05
Epoch 00092: val_loss improved from 0.00047 to 0.00047, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 502us/sample - loss: 1.0313e-05 - val_loss: 4.6521e-04
Epoch 93/128
5696/5734 [============================>.] - ETA: 0s - loss: 9.0439e-06
Epoch 00093: val_loss improved from 0.00047 to 0.00045, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 493us/sample - loss: 9.0685e-06 - val_loss: 4.5254e-04
Epoch 94/128
5696/5734 [============================>.] - ETA: 0s - loss: 8.5827e-06
Epoch 00094: val_loss improved from 0.00045 to 0.00045, saving model to best_params.

5696/5734 [============================>.] - ETA: 0s - loss: 3.9459e-06- ETA: 0s - loss: 3.6288e
Epoch 00121: val_loss did not improve from 0.00029
5734/5734 [==============================] - 3s 502us/sample - loss: 3.9383e-06 - val_loss: 2.9435e-04
Epoch 122/128
5696/5734 [============================>.] - ETA: 0s - loss: 4.7901e-06- ETA: 2s - loss: - 
Epoch 00122: val_loss improved from 0.00029 to 0.00029, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 500us/sample - loss: 4.7788e-06 - val_loss: 2.8637e-04
Epoch 123/128
5696/5734 [============================>.] - ETA: 0s - loss: 2.8901e-06- ETA: 2 - ETA: 1s - loss: 2.9018e-0 - ETA: 1s - loss: 2.8897e- - ETA: 0s - 
Epoch 00123: val_loss improved from 0.00029 to 0.00028, saving model to best_params.hdf5
5734/5734 [==============================] - 3s 499us/sample - loss: 2.8956e-06 - val_loss: 2.8313e-04
Epoch 124/128
5696/5734 [============================>.] - ETA: 0s - loss: 4.0804e-06
Epoch 00124

In [ ]:
# ma_10_2017 = MA(df[START_OF_2017 - 10 : START_OF_2018]["Close"], timeperiod=10, matype=0)
# ma_30_2017 = MA(df[START_OF_2017 - 30 : START_OF_2018]["Close"], timeperiod=30, matype=0)
# df_2017_neg8 = df[START_OF_2017 - 8 : START_OF_2018]
# k_2017, d_2017 = STOCH(df_2017_neg8["High"], df_2017_neg8["Low"], df_2017_neg8["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

In [24]:
# def removeNanFromNumpyArray(numpy_array):
#     temp = numpy_array.copy()
#     temp = temp[np.logical_not(np.isnan(temp))]
#     return temp
# print(len(ma_10_train))
# print(len(ma_30_train))
# print(len(k_train))
# print(len(d_train))

5793
5793
5793
5793


In [25]:
# ma_10_train = removeNanFromNumpyArray(ma_10_train)
# ma_30_train = removeNanFromNumpyArray(ma_30_train)
# k_train = removeNanFromNumpyArray(k_train)
# d_train = removeNanFromNumpyArray(d_train)

In [26]:
# def normalizeNumpyArray(numpy_array):
#     min_val = min(numpy_array)
#     max_val = max(numpy_array)
#     normalize_equation = lambda x: (x - min_val)/(max_val - min_val)
#     normalized_numpy_array = np.array([normalize_equation(x) for x in numpy_array])
#     return normalized_numpy_array
    

In [27]:
# ma_10_train = normalizeNumpyArray(ma_10_train)
# ma_30_train = normalizeNumpyArray(ma_30_train)
# k_train = normalizeNumpyArray(k_train)
# d_train = normalizeNumpyArray(d_train)

In [28]:
# print(len(ma_10_train))
# print(len(ma_30_train))
# print(len(k_train))
# print(len(d_train))
# print(df_train.shape)

5784
5764
5785
5785
(5793, 7)


In [29]:
# ma_10_train = ma_10_train[20:]
# k_train = k_train[21:]
# d_train = d_train[21:]

In [30]:
# print(len(ma_10_train))
# print(len(ma_30_train))
# print(len(k_train))
# print(len(d_train))

5764
5764
5764
5764
